# Rewriting with H-boxes

A useful additional generator for ZX-diagrams is the H-box. This allows us to more easily represent multilinear operators like the Toffoli or CCZ. With the H-box comes several additional rewrite rules.

In [ ]:
import sys, os; sys.path.insert(0, '..')
import pyzx as zx
import random
from fractions import Fraction
%config InlineBackend.figure_format = 'svg'

Z = zx.VertexType.Z
X = zx.VertexType.X
B = zx.VertexType.BOUNDARY
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

# Graphical Fourier Transform
We can transform H-boxes into networks of phase gadgets. This allows us to for instance decompose a diagram representing a CCT (a doubly controlled T gate) using phase gadgets.

In [ ]:
g = zx.Graph()
qs = 3
ins  = [g.add_vertex(B, qubit=i, row=0) for i in range(qs)]
zs   = [g.add_vertex(Z, qubit=i, row=2) for i in range(qs)]
outs = [g.add_vertex(B, qubit=i, row=4) for i in range(qs)]

hb = g.add_vertex(VertexType.H_BOX, qubit=-1, row=3)
g.set_phase(hb, Fraction(1,4))
for i in range(qs):
    g.add_edge((ins[i], zs[i]))
    g.add_edge((zs[i], outs[i]))
    g.add_edge((hb, zs[i]))
zx.draw(g, labels=True)

In [ ]:
from pyzx.fourier import *
g1 = g.copy()
print(check_fourier(g1, hb))
fourier(g1, hb)
zx.draw(g1)

Similarly, we can apply an inverse Fourier transform to change a phase gadget into a network of H-boxes.

In [ ]:
g = zx.Graph()
qs = 3
ins  = [g.add_vertex(B, qubit=i, row=0) for i in range(qs)]
zs   = [g.add_vertex(Z, qubit=i, row=2) for i in range(qs)]
outs = [g.add_vertex(B, qubit=i, row=4) for i in range(qs)]

v1 = g.add_vertex(VertexType.Z, qubit=-2, row=3)
v2 = g.add_vertex(VertexType.X, qubit=-1, row=3)
g.add_edge((v1,v2))
g.set_phase(v1, Fraction(1,4))
for i in range(qs):
    g.add_edge((ins[i], zs[i]))
    g.add_edge((zs[i], outs[i]))
    g.add_edge((v2, zs[i]))
zx.draw(g, labels=True)

In [ ]:
g1 = g.copy()
print(check_ifourier(g1, v1))
ifourier(g1, v1)
zx.draw(g1)

# Hyperpivot simplification

In the same way that we can use local complementation and pivoting to simplify ZX-diagrams and remove spiders with a Clifford phase from the diagram, we can also apply a hyperpivot rule on a hypergraph built using H-boxes. In this notebook we demonstrate this rewrite rules, and show that it suffices to reduce a class of benchmark circuits to a type of normal form.
For more information on how the hyperpivot works, see [this paper](https://arxiv.org/abs/2003.13564).

In [ ]:
ccz = zx.qasm("""
qreg q[3];
ccz q[0],q[1],q[2];
""").to_graph(zh=True)
display(zx.draw(ccz))
print(ccz.to_matrix())

In [ ]:
g = zx.qasm("""
qreg q[3];

ccz q[0],q[1],q[2];
h q[2];
t q[2];
ccz q[0],q[1],q[2];
h q[2];
t q[1];
ccz q[0],q[1],q[2];
s q[2];
ccx q[0],q[1],q[2];
""").to_graph(zh=True)
zx.draw(g, labels=True)

In [ ]:
h = g.copy()
zx.simplify.spider_simp(h)
zx.hsimplify.to_hypergraph_form(h)
zx.draw(h,labels=True)
print(zx.hsimplify.just_hpivot_simp(h))
zx.draw(h,labels=True)

The following strategy `hpivot_simp` combines hyperpivots with spider fusions in order to simplify the diagram as much as possible.

In [ ]:
h = g.copy()
zx.hsimplify.hpivot_simp(h)
zx.draw(h, labels=True)

We see that in this case we have no internal spiders left: every spider is connected to a boundary. It turns out that this behaviour holds for many circuits as we will also see below.

In [ ]:
zx.compare_tensors(g,h)

In [ ]:
dir_fast_circuits = os.path.join('..', 'circuits', 'Fast')

In [ ]:
d = os.path.join('..', 'circuits', 'Fast')
print('Circuit'.ljust(30) + '  qubits' + '   gates' + '    Z' + '       H' + '  reduced')
for f in os.listdir(d):
    f1 = os.path.join(d,f)
    if f.find('QFTAdd8') != -1: continue # takes too long
    if not os.path.isfile(f1) or f.find('before') == -1: continue
    print(f.ljust(30), end='')
    
    c = zx.Circuit.load(f1)
    for g in c.gates:
        if isinstance(g, zx.gates.ZPhase):
            g.phase = g.phase.limit_denominator(3628800)
        
    print(str(c.qubits).rjust(8), end='')
    print(str(len(c.gates)).rjust(8), end='')
    g = c.to_graph(zh=True)
    zx.hsimplify.hpivot_simp(g)
    g.normalize()
    z = len([v for v in g.vertices() if g.type(v) == 1])
    h = len([v for v in g.vertices() if g.type(v) == 3])
    print(str(z).rjust(5), end='')
    print(str(h).rjust(8), end='  ')
    
    print(g.qubit_count() * 2 == z)